In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchvision import datasets, transforms
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import sys
import copy
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
import pandas as pd
import pickle



# 하이퍼파라미터 설정
batch_size = 100
epochs = 10
lr = 0.01
PCA_dim = 8
CLS_num = 2



with open('./data/data.pkl','rb') as file:
    data = pickle.load(file)
X = data['X']
y = data['Y']



x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=70)



def Fit_to_quantum(X,PCA_dim):
    pca = PCA(n_components=PCA_dim)
    X_pca = pca.fit_transform(X)
    return X_pca
    


# 정규화 (표준 스케일러 사용)
#x_train_pca = Fit_to_quantum(x_train,PCA_dim)
#x_test_pca = Fit_to_quantum(x_test,PCA_dim)

# PyTorch Tensor로 변환
x_train_pca, y_train = torch.tensor(x_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)
x_test_pca, y_test = torch.tensor(x_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)


class Feature_data_loader(Dataset):
    def __init__(self,x_train,y_train):
        self.feature1 = x_train
        temp = copy.deepcopy(x_train)
        shuffle = torch.randperm(len(temp))
        self.feature2 = temp[shuffle]
        
        self.y1 = y_train
        temp_y = copy.deepcopy(y_train)
        self.y2 = temp_y[shuffle]
        
    def __len__(self):
        return len(self.feature1)
    def __getitem__(self,idx):
        input1 = self.feature1[idx]
        input2 = self.feature2[idx]
        if self.y1[idx] == self.y2[idx]:
            label = torch.tensor(1.).float()
        else:
            label = torch.tensor(0.).float()
        return [input1,input2],label


# DataLoader 생성


train_loader = DataLoader(TensorDataset(x_train_pca, y_train.float()), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(x_test_pca, y_test.float()), batch_size=batch_size, shuffle=False)


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from collections import OrderedDict
import math
from kan import KAN
from functions.training import Early_stop_train




class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.cls_layer_1 = nn.Linear(PCA_dim,PCA_dim*PCA_dim)
        self.cls_layer_2 = nn.Linear(PCA_dim*PCA_dim,PCA_dim*PCA_dim)
        self.cls_layer_3 = nn.Linear(PCA_dim*PCA_dim,PCA_dim*2-1)
        self.output_layer = nn.Linear(2*PCA_dim-1,1)
    def forward(self, x):
        x = self.cls_layer_1(x)
        x = nn.ReLU()(x)
        x = self.cls_layer_2(x)
        x = nn.ReLU()(x)
        x = self.cls_layer_3(x)
        x = nn.ReLU()(x)
        output = self.output_layer(x)
        output = nn.Sigmoid()(output)
        output = torch.squeeze(output)
        return output
# 모델, 손실 함수, 최적화 설정



model = Model(); criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)




train_process = Early_stop_train(model, optimizer, criterion)
train_process.train_model(train_loader,test_loader,epochs=5000,res=15)

_,acc = train_process.test(test_loader)
print(f"Test Accuracy: {acc:.2f}")




Test set: Average loss: 0.6804, Accuracy: 33.99999976158142/60 (57%)
Epoch 1 Loss 0.692297 acc : 0.495000 stop count : 0
Test set: Average loss: 0.6784, Accuracy: 33.99999976158142/60 (57%)
Epoch 2 Loss 0.696468 acc : 0.465000 stop count : 0
Test set: Average loss: 0.6763, Accuracy: 33.99999976158142/60 (57%)
Epoch 3 Loss 0.688588 acc : 0.502500 stop count : 0
Test set: Average loss: 0.6743, Accuracy: 33.99999976158142/60 (57%)
Epoch 4 Loss 0.688293 acc : 0.487500 stop count : 0
Test set: Average loss: 0.6723, Accuracy: 33.99999976158142/60 (57%)
Epoch 5 Loss 0.689496 acc : 0.480000 stop count : 0
Test set: Average loss: 0.6703, Accuracy: 33.99999976158142/60 (57%)
Epoch 6 Loss 0.692362 acc : 0.450000 stop count : 0
Test set: Average loss: 0.6682, Accuracy: 33.99999976158142/60 (57%)
Epoch 7 Loss 0.689492 acc : 0.465000 stop count : 0
Test set: Average loss: 0.6662, Accuracy: 33.99999976158142/60 (57%)
Epoch 8 Loss 0.688750 acc : 0.457500 stop count : 0
Test set: Average loss: 0.6642,

In [3]:
from kan import KAN
from functions.training import Early_stop_train_KAN
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.KAN = KAN([PCA_dim,PCA_dim*2-1,1],grid=1)
    def forward(self, x):
        output = self.KAN(x)
        output = nn.Sigmoid()(output)
        output = torch.squeeze(output)
        return output


model = Model(); criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)




train_process = Early_stop_train_KAN(model, optimizer, criterion)
train_process.train_model(train_loader,test_loader,epochs=100,res=25,lamb=0.05)

_,acc = train_process.test(test_loader)
print(f"Test Accuracy: {acc:.2f}")


Test set: Average loss: 0.4553, Accuracy: 38.999998569488525/60 (65%)
Epoch 1 Loss 2.311667 acc : 0.755000 reg : 1.799404 stop count : 0
Test set: Average loss: 0.3056, Accuracy: 52.99999952316284/60 (88%)
Epoch 2 Loss 2.071608 acc : 0.932500 reg : 1.570769 stop count : 0
Test set: Average loss: 0.2280, Accuracy: 56.99999928474426/60 (95%)
Epoch 3 Loss 1.966883 acc : 0.952500 reg : 1.676537 stop count : 0
Test set: Average loss: 0.1922, Accuracy: 57.999998331069946/60 (97%)
Epoch 4 Loss 1.920910 acc : 0.935000 reg : 1.742276 stop count : 0
Test set: Average loss: 0.1756, Accuracy: 57.999998331069946/60 (97%)
Epoch 5 Loss 1.858373 acc : 0.907500 reg : 1.703370 stop count : 0
Test set: Average loss: 0.1675, Accuracy: 57.999998331069946/60 (97%)
Epoch 6 Loss 1.763843 acc : 0.900000 reg : 1.467823 stop count : 0
Test set: Average loss: 0.1629, Accuracy: 57.999998331069946/60 (97%)
Epoch 7 Loss 1.722744 acc : 0.892500 reg : 1.453735 stop count : 0
Test set: Average loss: 0.1605, Accuracy: 